# Agenda

- 1. Creation of a Simple Python Lambda 
    - That does Regex Capture
- 2. Deploy the Lambda with API Gateway using SAM Template
    - Deploy it in local and evaluate result 
- 3. Adding a Custom Lambda Authorizer
    - Editing the SAM Template
    - Adding an Authorizer Lambda
- 4. Deploying the Final SAM Template
    - In Local
    - To AWS Cloud
- 5. Hit the API Endpoint